In [54]:
def seconds_to_minutes(seconds):
    minutes = seconds // 60  # Integer division
    remaining_seconds = seconds % 60

    if remaining_seconds < 10:
        return f"{int(minutes)}.0{int(remaining_seconds)}"
    return f"{int(minutes)}.{int(remaining_seconds)}"

# Example usage
seconds = 486
minutes = seconds_to_minutes(seconds)
print(f"{seconds} seconds is equal to {minutes}")

486 seconds is equal to 8.06 minutes


In [61]:
import os
from pytubefix import YouTube
from modules.youtube import extract_youtube_video_id, get_video_metadata

#video_url = "https://youtu.be/JhU0yO43b6o?si=mfsiB2wajdY55US9"

video_url = "https://youtu.be/x7X9w_GIm1s?si=QyrwdVD2lVmeontV"
video_id = extract_youtube_video_id(video_url)
video_meta = get_video_metadata(video_url)

audio_filename = video_meta['name']
audio_filepath = os.path.join("..", "transcript_audios", audio_filename)
yt = YouTube(url=f"https://www.youtube.com/watch?v={video_id}")
yt.streams

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="313" mime_type="video/webm" res="2160p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="401" mime_type="video/mp4" res="2160p" fps="30fps" vcodec="av01.0.12M.08" progressive="False" type="video">, <Stream: itag="271" mime_type="video/webm" res="1440p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="400" mime_type="video/mp4" res="1440p" fps="30fps" vcodec="av01.0.12M.08" progressive="False" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">, <Stream: itag="136" mi

In [62]:
download = yt.streams.get_audio_only().download(mp3=True, filename=audio_filepath)

In [63]:
download

'/Users/sudoleg/Projects/Personal/ytai/scripts/../transcript_audios/Python in 100 Seconds.mp3'

## Generate transcript via API

In [21]:
from openai import OpenAI

api_key = os.getenv("OPENAI_YTAI_API_KEY")
client = OpenAI(api_key=api_key)

audio_file= open(download, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="verbose_json",
  timestamp_granularities=['segment']
)

transcription_segments = [segment for segment in transcription.segments]
transcription_text = transcription.text

## Generate transcript locally

In [64]:
import whisper
import time

# Start the timer
start_time = time.time()

model = whisper.load_model("base")
transcription = model.transcribe(download)

# End the timer
end_time = time.time()

# Calculate the elapsed time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

transcription_segments = [segment for segment in transcription['segments']]
transcription_text = transcription['text']

/Users/sudoleg/Projects/Personal/ytai/.venv/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, m

Execution time: 8.90 seconds


In [65]:
for segment in transcription_segments:
    start = seconds_to_minutes(float(segment['start']))
    end = seconds_to_minutes(float(segment['end']))
    print(f"Timemstamp: {start} - {end}")
    print(segment['text'])
    print("---")

Timemstamp: 0.00 - 0.05
 Python, a high-level, interpreted programming language famous for its zen-like code.
---
Timemstamp: 0.05 - 0.09
 It's arguably the most popular language in the world because it's easy to learn,
---
Timemstamp: 0.09 - 0.14
 yet practical for serious projects. In fact, you're watching this YouTube video in a Python web
---
Timemstamp: 0.14 - 0.19
 application right now. It was created by Kwiidovan Rossum and released in 1991, who named it after
---
Timemstamp: 0.19 - 0.25
 Monty Python's blind circus, which is why you'll sometimes find spam and eggs instead of food and bar
---
Timemstamp: 0.25 - 0.30
 in code samples. It's commonly used to build server-side applications, like web apps with the Django
---
Timemstamp: 0.30 - 0.35
 framework, and is the language of choice for big data analysis and machine learning. Many students
---
Timemstamp: 0.35 - 0.40
 choose Python to start learning to code because of its emphasis on readability as outlined by the zen
---
Tim

In [66]:
from modules.helpers import save_response_as_file

save_response_as_file(dir_name="../transcribed", filename=video_meta['name'], file_content=transcription.text)

AttributeError: 'dict' object has no attribute 'text'